# Classical Forecasting Methods: Holt-Winter's Model

**Approximate Learning Time**: Up to 2 hours

--- 

## Holt-Winters Model

The Holt-Winters model is a method used for forecasting time series data that shows both **trend** and **seasonality**. It works by applying three smoothing techniques:

1. **Level smoothing**: Captures the overall average of the series.
2. **Trend smoothing**: Captures the direction and rate of change in the data over time.
3. **Seasonal smoothing**: Captures repeating patterns (seasonality) in the data at regular intervals.

By combining these three components, Holt-Winters can make predictions that account for both long-term trends and short-term seasonal fluctuations. In simple terms, it’s like saying: "The model predicts the future by looking at the average, the trend over time, and any repeating patterns in the data." There are several simplications of this model, which are all broadly termed as **Exponential Smoothing models** ([Wikipedia](https://en.wikipedia.org/wiki/Exponential_smoothing)). 

**Note**: These models are designed for univariate time series, meaning it models one time series at a time. Therefore, we will build individual univariate models for each of the time series in our dataset. To handle **multivariate time series**, where multiple variables are modeled together, we will introduce the **Vector Autoregression (VAR)** approach in the next notebook.

--- 

Let's load the log daily returns of exchange rates, and split the data into train, validation, and test subsets!


In [ ]:
import pathlib
import numpy as np
import pandas as pd
from termcolor import colored

import sys; sys.path.append("../")
import utils

from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing

# To avoid flooding of the screen with convergence warnings during hyperparameter tuning
import warnings
warnings.filterwarnings("ignore")

## WARNING: To compare different models on the same horizon, keep this same across the notebooks 
FORECASTING_HORIZON = [4, 8, 12] # weeks 
MAX_FORECASTING_HORIZON = max(FORECASTING_HORIZON)

SEQUENCE_LENGTH = 2 * MAX_FORECASTING_HORIZON
PREDICTION_LENGTH = MAX_FORECASTING_HORIZON

DIRECTORY_PATH_TO_SAVE_RESULTS = pathlib.Path('../results/DIY/').resolve()
MODEL_NAME = "ExpSmooth"

RESULTS_DIRECTORY = DIRECTORY_PATH_TO_SAVE_RESULTS / MODEL_NAME
if RESULTS_DIRECTORY.exists():
    print(colored(f'Directory {str(RESULTS_DIRECTORY)} already exists.'
           '\nThis notebook will overwrite results in the same directory.'
           '\nYou can also create a new directory if you want to keep this directory untouched.'
           ' Just change the `MODEL_NAME` in this notebook.\n', "red" ))
else:
    RESULTS_DIRECTORY.mkdir(parents=True)


# load data

data, transformed_data = utils.load_tutotrial_data(dataset='exchange_rate', log_transform=True)
data = transformed_data

train_val_data = data.iloc[:-MAX_FORECASTING_HORIZON]
train_data, val_data = train_val_data.iloc[:-MAX_FORECASTING_HORIZON], train_val_data.iloc[-MAX_FORECASTING_HORIZON:]
test_data = data.iloc[-MAX_FORECASTING_HORIZON:]

print(f"Number of steps in training data: {len(train_data)}\nNumber of steps in validation data: {len(val_data)}\nNumber of steps in test data: {len(test_data)}")

%load_ext autoreload
%autoreload 2

--- 

## Hyperparameter Tuning

These models are provided in `statsmodels` library. Specifically, they are provided through `SimpleExpSmoothing` ([documentation](https://www.statsmodels.org/stable/generated/statsmodels.tsa.holtwinters.SimpleExpSmoothing.html)) and a more general `ExponentialSmoothing` ([documentation](https://www.statsmodels.org/dev/generated/statsmodels.tsa.holtwinters.ExponentialSmoothing.html)) functions. 

In this tutorial, we will run a hyperparameter search on the values of `seasonal_period`.

In [2]:
s_values = [0, 2, 4, 8, 12]
best_mase, best_mase_model = {}, {}
for col in train_data.columns:
    best_mase[col] = np.inf
    best_mase_model[col] = {}

    for s in s_values:
        if s <= 1: # Holt's Linear Trend Model
            model = SimpleExpSmoothing(train_data[col], initialization_method="estimated")
        else: # Holt-Winter's Model
            model = ExponentialSmoothing(train_data[col], trend='add', seasonal='add', seasonal_periods=s, initialization_method="estimated")
        
        model = model.fit()
        y_pred = model.forecast(len(val_data))
        mase = utils.mean_absolute_scaled_error(y_pred.values, val_data[col].values, train_data[col].values)
        
        if mase < best_mase[col]:
            best_mase[col] = mase
            best_mase_model[col] = (model, model.aic, s)

In [ ]:
print("Best paramerters")
for col in train_data.columns:
    mase = best_mase[col]
    s = best_mase_model[col][2]
    print(f"Col: {col}\tValidation-MASE:{mase: 0.3f}\tSeasonality:{s}")

---

## Refit on Train-Val Subset & Forecast

To measure the model's performance on the test data, we will first retrain the model using the combined train-validation dataset. Then, we will compute the MASE metric on the test dataset to evaluate its performance.
Additionally, we will store the test predictions for later comparison with other forecasting methods.

In [ ]:
test_predictions = {}
best_model_metrics = {}
for col in test_data.columns:
    best_model_metrics[col] = {}
    ts = train_val_data[col]
    
    # retrain the model with best mase parameters on train_val_data
    s = best_mase_model[col][2]
    if s <= 1: # Holt's Linear Trend Model
        model = SimpleExpSmoothing(ts, initialization_method="estimated")
    else: # Holt-Winter's Model
        model = ExponentialSmoothing(ts, trend='add',  seasonal='add', seasonal_periods=s, initialization_method="estimated")
    
    model = model.fit()

    # get metrics and predictions
    predictions = model.forecast(len(test_data))
    test_predictions[f"{MODEL_NAME}_{col}_mean"] = predictions.values

test_predictions_df = pd.DataFrame(test_predictions, index=test_data.index)

test_predictions_df.to_csv(f"{str(RESULTS_DIRECTORY)}/predictions.csv", index=True)
print(test_predictions_df.shape)
test_predictions_df.head()
    

---

## Evaluate

Let's compute the metrics by comparing the predictions with that of the target data. Note that we will have to rename the columns of the dataframe to match the expected column names by the function. 

In [ ]:
# compute MASE metrics
model_metrics, records = utils.get_mase_metrics(
    historical_data=train_val_data,
    test_predictions=test_predictions_df.rename(
            columns={x:x.split("_")[1] for x in test_predictions_df.columns
        }),
    target_data=test_data,
    forecasting_horizons=FORECASTING_HORIZON,
    columns=data.columns, 
    model_name=MODEL_NAME,
)

records = pd.DataFrame(records)

records.to_csv(f"{str(RESULTS_DIRECTORY)}/metrics.csv", index=False)
records[['col', 'horizon', 'mase']].pivot(index=['horizon'], columns='col')


---

## Compare Models

In [ ]:
utils.display_results(path=DIRECTORY_PATH_TO_SAVE_RESULTS, metric='mase')

---

## Plot Forecasts

In [ ]:
fig, axs = utils.plot_forecasts(
    historical_data=train_val_data,
    forecast_directory_path=DIRECTORY_PATH_TO_SAVE_RESULTS,
    target_data=test_data,
    columns=data.columns,
    n_history_to_plot=10, 
    forecasting_horizon=MAX_FORECASTING_HORIZON,
    dpi=200,
    plot_se=False,
)

--- 

## Conclusions

We learned about exponential smoothing models, searched for the best seasonal period for 8 time series in the dataset, and finally, evalauted its performance using the MASE metric. We compared it with ARIMA models' performance in the previous notebook. 

--- 

## Exercises

- Perform a wider hyperparameter search on the parameters for `ExponentialSmoothing` ([documentation](https://www.statsmodels.org/dev/generated/statsmodels.tsa.holtwinters.ExponentialSmoothing.html))

- Apply a normalization procedure (e.g., **min-max scaling**) to the data, ensuring that only the training data is used for fitting the scaler. Perform the modeling process on the normalized data and, after generating the final model's predictions, invert the normalization to return the output to its original scale. See `sklearn.preprocessing.MinMaxScaler` ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html))

- Additionally, perform the modeling on the **raw data**, without applying any transformation (such as converting it into log daily returns), to compare results directly with the untransformed dataset.

---

## Next Steps

- To learn about Vector Autoregression method to model multivariate time series, proceed to notebook 3.4

- To learn about other machine learning based approaches, check out the module 4 (XGBoost), module 5 (LSTM-based models), module 6 (Transformer based models), or module 7 (LLM-based models).
---